In [1]:
import geetiles
import os
import geopandas as gpd
from geetiles import cmds
from google.cloud import storage
import numpy as np
from progressbar import progressbar as pbar
from joblib import Parallel, delayed


In [3]:
chipsets_folder = "/opt/earthdata/chipsets"
chipsets_files = [f for f in os.listdir(chipsets_folder) if f.endswith(".geojson")]
dataset = 's1grd-2022'

# add groups to chipsets

In [105]:
def add_groups(chipsets_folder, chipset_file):
    if chipset_file.startswith('38a2a521d3489_partitions_aschips_150c70b88afea.geojson'):
        return
        
    z = gpd.read_file(f"{chipsets_folder}/{chipset_file}")
    z['group'] = [str(i) for i in np.random.randint(20, size=len(z))]
    z.to_file(f"{chipsets_folder}/{chipset_file}", driver='GeoJSON')

_ = Parallel(verbose=10, n_jobs=-1)(delayed(add_groups)(chipsets_folder, chipset_file) for chipset_file in chipsets_files)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    6.7s
[Parallel(n_jobs=-1)]: Done  49 tasks      | elapsed:    8.7s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:   10.8s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:   13.4s
[Parallel(n_jobs=-1)]: Done  98 tasks      | elapsed:   15.0s
[Parallel(n_jobs=-1)]: Done 117 tasks      | elapsed:   16.4s
[Parallel(n_jobs=-1)]: Done 136 tasks      | elapsed:   18.3s
[Parallel(n_jobs=-1)]: Done 157 tasks      | elapsed:   21.2s
[Parallel(n_jobs=-1)]: Done 178 tasks      | elapsed:   23.7s
[Parallel(n_jobs=-1)]: Done 201 tasks      | elapsed:   26.7s
[Parallel(n_jobs=-1)]: Done 224 tasks      | elapsed:   29.2s
[Parallel(n_jobs=-1)]: Done 249 tasks      | elapsed:   31.4s
[Parallel(n_jobs=-1)]: Done 274 tasks      | elapsed:  

## move away files not in groups 0,1

In [4]:
dest_dir = "/opt/earthdata/other_groups"


In [6]:
def move_files_notin_groups01(chipsets_folder, chipset_file):
    c = 0
    z = gpd.read_file(f"{chipsets_folder}/{chipset_file}")
    dataset_folder = f"{chipsets_folder}/{chipset_file.split('.')[0]}/{dataset}"

    dest_dataset_folder = f"{dest_dir}/{chipset_file.split('.')[0]}/{dataset}"

    for identifier in z.identifier.values: #[~z.group.isin(['0','1'])].identifier.values:
        chip_file = f"{dataset_folder}/{identifier}.tif"
        if os.path.isfile(chip_file):
            os.makedirs(dest_dataset_folder, exist_ok=True)
            os.rename(chip_file, f"{dest_dataset_folder}/{identifier}.tif")
            c += 1
    return c
    
counts = Parallel(verbose=10, n_jobs=-1)(delayed(move_files_notin_groups01)(chipsets_folder, chipset_file) for chipset_file in chipsets_files)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done  49 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:    6.2s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:    7.3s
[Parallel(n_jobs=-1)]: Done  98 tasks      | elapsed:    8.1s
[Parallel(n_jobs=-1)]: Done 117 tasks      | elapsed:    8.7s
[Parallel(n_jobs=-1)]: Done 136 tasks      | elapsed:    9.4s
[Parallel(n_jobs=-1)]: Done 157 tasks      | elapsed:   10.6s
[Parallel(n_jobs=-1)]: Done 178 tasks      | elapsed:   11.6s
[Parallel(n_jobs=-1)]: Done 201 tasks      | elapsed:   12.9s
[Parallel(n_jobs=-1)]: Done 224 tasks      | elapsed:   13.9s
[Parallel(n_jobs=-1)]: Done 249 tasks      | elapsed:   14.8s
[Parallel(n_jobs=-1)]: Done 274 tasks      | elapsed:  

In [7]:
sum(counts)

35938

In [156]:
z = gpd.read_file("/opt/earthdata/chipsets/004c26065c22b_partitions_aschips_0a4e74364931c.geojson")

In [157]:
z[z.group.isin(['0','1'])].explore()